# GA4 Sessions Analysis at DMA Level

This notebook analyzes Google Analytics 4 (GA4) sessions data grouped at the Designated Market Area (DMA) level.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import sys
import os

# Set up plotting
plt.style.use('ggplot')
sns.set(style="whitegrid")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## 1. Load the GA4 Sessions Data

In [2]:
# Load GA4 sessions data
ga4_df = pd.read_csv('../raw_data/ga4_sessions.csv')

# Display the first few rows
print(f"Total GA4 records: {len(ga4_df)}")
ga4_df.head()

Total GA4 records: 100000


,Date,Region,Region ID,City,City ID,Sessions,Engaged sessions
0,20250319,(not set),(not set),(not set),(not set),19,1
1,20250319,Alabama,US-AL,Headland,1013037,1,0
2,20250319,Alabama,US-AL,Huntsville,1013042,1,0
3,20250319,Alabama,US-AL,Uniontown,9207090,1,1
4,20250319,Alaska,US-AK,Sitka,1012913,1,0


In [3]:
# Check for missing values
print("Missing values in each column:")
ga4_df.isnull().sum()

Missing values in each column:


Date                0
Region              0
Region ID           0
City                0
City ID             0
Sessions            0
Engaged sessions    0
dtype: int64

In [4]:
# Check data types
ga4_df.dtypes

Date                 int64
Region              object
Region ID           object
City                object
City ID             object
Sessions             int64
Engaged sessions     int64
dtype: object

In [5]:
# Convert Date to proper datetime format (assuming YYYYMMDD format)
ga4_df['Date'] = pd.to_datetime(ga4_df['Date'], format='%Y%m%d')

# Handling potential '(not set)' values
not_set_mask = (ga4_df['Region'] == '(not set)') | (ga4_df['City'] == '(not set)')
print(f"Records with '(not set)' for Region or City: {not_set_mask.sum()}")

Records with '(not set)' for Region or City: 4232


## 2. Load the Geo Spine Data

In [6]:
# Load geo spine data - importantly reading zip_code as string to preserve leading zeros
geo_spine_df = pd.read_csv('../data/reference/geo_spine.csv', dtype={'zip_code': str})

# Display the first few rows
print(f"Total geo spine records: {len(geo_spine_df)}")
geo_spine_df.head()

Total geo spine records: 40775


,zip_code,dma_code,dma_name,city,state,state_name,geo_key_zip,geo_key_city,geo_key_dma,geo_key_state
0,11005,501,NEW YORK,FLORAL PARK,NY,New York,11005,"FLORAL PARK, NY",NEW YORK,NY
1,11010,501,NEW YORK,FRANKLIN SQUARE,NY,New York,11010,"FRANKLIN SQUARE, NY",NEW YORK,NY
2,11020,501,NEW YORK,GREAT NECK,NY,New York,11020,"GREAT NECK, NY",NEW YORK,NY
3,11021,501,NEW YORK,GREAT NECK,NY,New York,11021,"GREAT NECK, NY",NEW YORK,NY
4,11023,501,NEW YORK,GREAT NECK,NY,New York,11023,"GREAT NECK, NY",NEW YORK,NY


In [7]:
# Verify data types - especially that zip_code is string
geo_spine_df.dtypes

zip_code         object
dma_code          int64
dma_name         object
city             object
state            object
state_name       object
geo_key_zip       int64
geo_key_city     object
geo_key_dma      object
geo_key_state    object
dtype: object

In [8]:
# Prepare city mapping from geo_spine for joining
# Extract state from geo_key_city (format: "CITY, STATE")
geo_spine_df['city_cleaned'] = geo_spine_df['city'].str.upper()
geo_spine_df['state_upper'] = geo_spine_df['state'].str.upper()

# Create a dictionary mapping (city, state) to DMA
city_to_dma = geo_spine_df.groupby(['city_cleaned', 'state_upper'])['dma_name'].first().reset_index()
print(f"Number of unique city-state combinations: {len(city_to_dma)}")
city_to_dma.head()

Number of unique city-state combinations: 24101


,city_cleaned,state_upper,dma_name
0,AARONSBURG,PA,JOHNSTOWN-ALTOONA-ST COLGE
1,ABBEVILLE,AL,DOTHAN
2,ABBEVILLE,GA,MACON
3,ABBEVILLE,LA,"LAFAYETTE, LA"
4,ABBEVILLE,MS,MEMPHIS


## 3. Join GA4 Data to Geo Spine

In [9]:
# Prepare GA4 data for joining
ga4_df['City_Upper'] = ga4_df['City'].str.upper()
ga4_df['State_Code'] = ga4_df['Region ID'].str.split('-').str[1]

# Join GA4 data with city-to-DMA mapping
ga4_geo_df = ga4_df.merge(
    city_to_dma,
    left_on=['City_Upper', 'State_Code'],
    right_on=['city_cleaned', 'state_upper'],
    how='left'
)

# Check the join success rate
join_success_rate = (ga4_geo_df['dma_name'].notna().sum() / len(ga4_geo_df)) * 100
print(f"Join success rate: {join_success_rate:.2f}%")

# Display the joined data
ga4_geo_df.head()

Join success rate: 84.12%


,Date,Region,Region ID,City,City ID,Sessions,Engaged sessions,City_Upper,State_Code,city_cleaned,state_upper,dma_name
0,2025-03-19,(not set),(not set),(not set),(not set),19,1,(NOT SET),NaN,NaN,NaN,NaN
1,2025-03-19,Alabama,US-AL,Headland,1013037,1,0,HEADLAND,AL,HEADLAND,AL,DOTHAN
2,2025-03-19,Alabama,US-AL,Huntsville,1013042,1,0,HUNTSVILLE,AL,HUNTSVILLE,AL,HUNTSVILLE-DECATUR (FLOR)
3,2025-03-19,Alabama,US-AL,Uniontown,9207090,1,1,UNIONTOWN,AL,UNIONTOWN,AL,MONTGOMERY-SELMA
4,2025-03-19,Alaska,US-AK,Sitka,1012913,1,0,SITKA,AK,SITKA,AK,JUNEAU


In [10]:
# Check records that didn't join successfully
unmatched_ga4 = ga4_geo_df[ga4_geo_df['dma_name'].isna()]
print(f"Number of unmatched GA4 records: {len(unmatched_ga4)}")
unmatched_ga4.head()

Number of unmatched GA4 records: 15879


,Date,Region,Region ID,City,City ID,Sessions,Engaged sessions,City_Upper,State_Code,city_cleaned,state_upper,dma_name
0,2025-03-19,(not set),(not set),(not set),(not set),19,1,(NOT SET),NaN,NaN,NaN,NaN
8,2025-03-19,Arizona,US-AZ,San Tan Valley,9052798,1,1,SAN TAN VALLEY,AZ,NaN,NaN,NaN
14,2025-03-19,California,US-CA,(not set),(not set),3,1,(NOT SET),CA,NaN,NaN,NaN
28,2025-03-19,California,US-CA,East Los Angeles,9051873,4,4,EAST LOS ANGELES,CA,NaN,NaN,NaN
41,2025-03-19,California,US-CA,Naval Air Station Point Mugu,1014152,1,1,NAVAL AIR STATION POINT MUGU,CA,NaN,NaN,NaN


In [11]:
# For unmatched records, try to use Region (State) level geographic data
# Create a state-to-DMA mapping (using the most common DMA in each state)
state_to_dma = geo_spine_df.groupby('state_upper')['dma_name'].agg(
    lambda x: x.value_counts().index[0] if len(x.value_counts()) > 0 else None
).reset_index()

# Apply state-level mapping for records without city match
state_mapping_dict = dict(zip(state_to_dma['state_upper'], state_to_dma['dma_name']))

# Fill missing DMA values with state-based mapping
ga4_geo_df.loc[ga4_geo_df['dma_name'].isna(), 'dma_name'] = \
    ga4_geo_df.loc[ga4_geo_df['dma_name'].isna(), 'State_Code'].map(state_mapping_dict)

# Check the updated join success rate
updated_join_rate = (ga4_geo_df['dma_name'].notna().sum() / len(ga4_geo_df)) * 100
print(f"Updated join success rate: {updated_join_rate:.2f}%")

Updated join success rate: 99.65%


## 4. Group Data at DMA Level

In [ ]:
# Add year-month field for temporal analysis
ga4_geo_df['year_month'] = ga4_geo_df['Date'].dt.to_period('M')

# Group by DMA and calculate metrics
dma_sessions = ga4_geo_df.groupby('dma_name').agg(
    total_sessions=('Sessions', 'sum'),
    engaged_sessions=('Engaged sessions', 'sum'),
    unique_cities=('City', 'nunique'),
    first_session_date=('Date', 'min'),
    last_session_date=('Date', 'max')
).reset_index()

# Calculate engagement rate
dma_sessions['engagement_rate'] = (dma_sessions['engaged_sessions'] / dma_sessions['total_sessions'] * 100).round(2)

# Calculate days between first and last session
dma_sessions['active_days'] = (dma_sessions['last_session_date'] - dma_sessions['first_session_date']).dt.days

# Sort by total sessions
dma_sessions = dma_sessions.sort_values('total_sessions', ascending=False)

# Display the results
print(f"Number of DMAs with sessions: {len(dma_sessions)}")
dma_sessions.head(10)

In [ ]:
# Group by DMA and year-month to see temporal trends
dma_monthly_sessions = ga4_geo_df.groupby(['dma_name', 'year_month']).agg(
    monthly_sessions=('Sessions', 'sum'),
    monthly_engaged=('Engaged sessions', 'sum')
).reset_index()

# Calculate monthly engagement rate
dma_monthly_sessions['monthly_engagement_rate'] = (
    dma_monthly_sessions['monthly_engaged'] / dma_monthly_sessions['monthly_sessions'] * 100
).round(2)

# Convert Period to string for plotting
dma_monthly_sessions['year_month_str'] = dma_monthly_sessions['year_month'].astype(str)

# Show sample of temporal data
dma_monthly_sessions.head()

## 5. Data Summary and Visualization

In [ ]:
# Summary statistics for DMA sessions
dma_sessions.describe()

In [ ]:
# Plot top 15 DMAs by total sessions
plt.figure(figsize=(14, 8))
ax = sns.barplot(x='total_sessions', y='dma_name', data=dma_sessions.head(15))
plt.title('Top 15 DMAs by Total Sessions', fontsize=16)
plt.xlabel('Total Sessions', fontsize=14)
plt.ylabel('DMA', fontsize=14)

# Add engagement rate as text
for i, row in enumerate(dma_sessions.head(15).itertuples()):
    ax.text(row.total_sessions + 5, i, f"{row.engagement_rate}%", va='center')
    
plt.tight_layout()
plt.show()

In [ ]:
# Plot distribution of sessions across all DMAs
plt.figure(figsize=(12, 6))
sns.histplot(dma_sessions['total_sessions'], bins=30, kde=True)
plt.title('Distribution of Total Sessions by DMA', fontsize=16)
plt.xlabel('Total Sessions', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.axvline(dma_sessions['total_sessions'].mean(), color='red', linestyle='--', label=f'Mean: {dma_sessions["total_sessions"].mean():.1f}')
plt.axvline(dma_sessions['total_sessions'].median(), color='green', linestyle='--', label=f'Median: {dma_sessions["total_sessions"].median():.1f}')
plt.legend()
plt.show()

In [ ]:
# Plot engagement rate by DMA (top 15)
plt.figure(figsize=(14, 8))
engagement_data = dma_sessions.head(15).sort_values('engagement_rate', ascending=False)
sns.barplot(x='engagement_rate', y='dma_name', data=engagement_data)
plt.title('Engagement Rate by DMA (Top 15 by Sessions)', fontsize=16)
plt.xlabel('Engagement Rate (%)', fontsize=14)
plt.ylabel('DMA', fontsize=14)
plt.axvline(dma_sessions['engagement_rate'].mean(), color='red', linestyle='--', label=f'Mean: {dma_sessions["engagement_rate"].mean():.1f}%')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Plot sessions over time for top 5 DMAs
top_5_dmas = dma_sessions.head(5)['dma_name'].tolist()
plt.figure(figsize=(16, 8))

for dma in top_5_dmas:
    dma_data = dma_monthly_sessions[dma_monthly_sessions['dma_name'] == dma]
    plt.plot(dma_data['year_month_str'], dma_data['monthly_sessions'], marker='o', linewidth=2, label=dma)

plt.title('Monthly Sessions Trend for Top 5 DMAs', fontsize=16)
plt.xlabel('Year-Month', fontsize=14)
plt.ylabel('Number of Sessions', fontsize=14)
plt.xticks(rotation=45)
plt.legend(title='DMA')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Plot engagement rate over time for top 5 DMAs
plt.figure(figsize=(16, 8))

for dma in top_5_dmas:
    dma_data = dma_monthly_sessions[dma_monthly_sessions['dma_name'] == dma]
    plt.plot(dma_data['year_month_str'], dma_data['monthly_engagement_rate'], marker='o', linewidth=2, label=dma)

plt.title('Monthly Engagement Rate Trend for Top 5 DMAs', fontsize=16)
plt.xlabel('Year-Month', fontsize=14)
plt.ylabel('Engagement Rate (%)', fontsize=14)
plt.xticks(rotation=45)
plt.legend(title='DMA')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Key Insights and Findings

In [ ]:
# Calculate market concentration metrics
total_market_sessions = dma_sessions['total_sessions'].sum()
dma_sessions['market_share'] = dma_sessions['total_sessions'] / total_market_sessions * 100

# Calculate cumulative market share
dma_sessions['cumulative_share'] = dma_sessions['market_share'].cumsum()

# Top 10 DMAs market share
top_10_share = dma_sessions.head(10)['market_share'].sum()
print(f"Top 10 DMAs account for {top_10_share:.1f}% of all sessions")

# Find the DMAs that make up 80% of sessions (Pareto principle check)
pareto_dmas = dma_sessions[dma_sessions['cumulative_share'] <= 80]
print(f"Number of DMAs accounting for 80% of sessions: {len(pareto_dmas)}")

# Display top DMAs with their market share
dma_sessions[['dma_name', 'total_sessions', 'engagement_rate', 'market_share', 'cumulative_share']].head(15)

In [ ]:
# Compare total sessions with engagement rate
plt.figure(figsize=(12, 8))
plt.scatter(dma_sessions['total_sessions'], dma_sessions['engagement_rate'], alpha=0.7)
plt.title('Engagement Rate vs. Total Sessions by DMA', fontsize=16)
plt.xlabel('Total Sessions', fontsize=14)
plt.ylabel('Engagement Rate (%)', fontsize=14)
plt.grid(True, alpha=0.3)

# Annotate top DMAs
for i, row in dma_sessions.head(10).iterrows():
    plt.annotate(row['dma_name'], 
                 xy=(row['total_sessions'], row['engagement_rate']),
                 xytext=(5, 5), textcoords='offset points',
                 fontsize=9)
    
plt.tight_layout()
plt.show()

## 7. Conclusion and Next Steps

In this analysis, we have:

1. Successfully joined the GA4 sessions data with geographic reference data
2. Grouped and analyzed the data at the DMA level
3. Identified the top DMAs by session volume and engagement rate
4. Examined temporal trends in session activity and engagement by DMA
5. Analyzed market concentration and distribution of sessions across DMAs

Next steps could include:

1. Deeper analysis of engagement patterns by DMA
2. Correlation of GA4 sessions with other metrics such as orders or marketing spend
3. Analysis of geographic factors that might influence engagement rates
4. Predictive modeling to forecast future traffic and engagement by DMA